# Final Project - Hypothesis Testing

After building up my data cleaning skills in the previous two assignments, this project gave me a chance to use my own cleaned data to draw conclusions about mean housing prices during a recession. This was my first opportunity to do a project from start-to-finish in this data science specialization. 

I analyzed GDP data and isolated periods when the United States fell into an economic recession. I also found the housing prices in university towns and non-university towns. I then conducted a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom (`price_ratio=quarter_before_recession/recession_bottom`)


<br>
*Hypotheses*

**Null Hypothesis**: University towns have their mean housing prices equivalently effected by recessions as non-university towns.

**Alternative Hypothesis**: University towns have their mean housing prices less effected by recessions than non-university towns.
<br>

<br>
The data was obtained from the links below:

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this project, I only needed to analyze GDP data from the first quarter of 2000 onward.
<br>

Helpful Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    data = pd.read_table('university_towns.txt', header=None)
    symbol = 'edit'

    # Create a column for the states each college town is located within.
    data['State'] = [a.split('[')[0] if symbol in a else np.nan for a in data[0]]
    data['State'].fillna(method='ffill', inplace=True)
    
    # Create county regions column, which omits the university name located in the county.
    data['RegionName'] = data[0].str.split('(', expand=True)[0]
    data['RegionName'] = data['RegionName'].str.split('[', expand=True)[0].str.strip()
    
    # Remove unneeded columns
    data.drop(0, axis=1, inplace=True)
    
    # Remove extra rows where a region name and its state name are the same
    data = data.query('State != RegionName')
    data.reset_index(drop=True, inplace=True)
    
    return data

get_list_of_university_towns().head(12)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [5]:
def get_gdp():
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp.rename(columns={'Unnamed: 4': 'Quarterly', 'Unnamed: 5': 'GDP'})

    #Select columns to keep
    gdp = gdp[[4, 5]]
    
    #Select rows from 2000q1 onward
    gdp = gdp[219:].reset_index(drop=True)
    return gdp

def get_recession_start_index():
    gdp = get_gdp()
    for i in range(len(gdp) - 2):
        if (gdp['GDP'].iloc[i] > gdp['GDP'].iloc[i+1] > gdp['GDP'].iloc[i+2]): 
            return i

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    i = get_recession_start_index()
    return gdp['Quarterly'].iloc[i]

get_recession_start()

'2008q3'

In [6]:
def get_recession_end_index():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    idx = get_recession_start_index()
    
    for idx in range(idx, len(gdp)-2):
        if(gdp['GDP'].iloc[idx+2] > gdp['GDP'].iloc[idx+1] > gdp['GDP'].iloc[idx]):
            return idx + 2
        

def get_recession_end():
    gdp = get_gdp()
    idx = get_recession_end_index()
    return gdp['Quarterly'].iloc[idx]


get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start_index()
    end = get_recession_end_index()
    
    gdp = get_gdp()
    
    for i in range(start, end):
        
        # Recession years
        recession = gdp[start:end]
        
        #Find data
        min_gdp = recession['GDP'].min()
        
        # Return lowest recession gdp and its quarter
        mini = recession.where(recession['GDP'] == min_gdp).dropna()
        
        return mini['Quarterly'].iloc[0]
               
get_recession_bottom()

'2009q2'

In [8]:
def get_a_house():
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
# Use this purely for referencing the original dataframe
    orig = house

    # Step 1) Mapping - Change State abbreviation to its full name 
    house['State'] = house['State'].map(states)

    # Create a mult-index
    house = house.set_index(['State', 'RegionName'])
    # Step 2) Reduce dataframe to 67 columns

    # FIND index for the first needed year to include in a new dataframe
    start = 0
    for i in house.columns:
        if i == '2000-01':
            break
        start += 1

    # End year to include in a new dataframe
    end = len(house.columns)

    # FIXME: Change 49 to start after fixing for loop
    house = house.iloc[:, start:end]

    return house

get_a_house().head(12)

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
Nevada,Las Vegas,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
California,San Diego,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
Texas,Dallas,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
California,San Jose,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700


In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = get_a_house()

    avg = house.groupby(np.arange(len(house.columns))//3, axis=1).mean()

    # Rename averaged columns
    count = 0
    quarter = 1
    year = 2000
    rename = []
    
    for i in avg:
        year_quarter = "{}q{}".format(year, quarter)
        rename.append(year_quarter)
        count = (count + 1) % 4
        quarter = (quarter % 4) + 1
        if count == 0:
            year += 1

    avg.columns = rename
    
    return avg

convert_housing_data_to_quarters().head(12)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,132600.000000,134366.666667,135400.000000,137000.000000,139533.333333,141733.333333,143366.666667,146133.333333,149333.333333,150933.333333,...,170066.666667,173400.000000,175466.666667,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,222900.000000,234366.666667,245433.333333,256033.333333,267200.000000,276266.666667,284500.000000,291933.333333,301233.333333,312866.666667,...,480200.000000,489033.333333,496433.333333,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,84466.666667,83866.666667,84866.666667,87833.333333,89733.333333,89300.000000,89066.666667,90900.000000,92566.666667,93800.000000,...,106633.333333,108900.000000,111533.333333,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,374266.666667,406566.666667,431866.666667,455500.000000,470666.666667,470200.000000,456800.000000,445566.666667,441433.333333,457766.666667,...,679400.000000,697033.333333,714933.333333,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [10]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    # Year before recession start
    gdp = get_gdp()
    s = get_recession_start_index()
    before = gdp['Quarterly'].iloc[s-1]

    # Year of recession bottom 
    bottom = get_recession_bottom()

    # List of all towns and their housing data per quarter
    towns = convert_housing_data_to_quarters()

    # Price ratio for t-test: price ratio = (quarter-before recession start)/(quarter of recession botton)
    towns['Ratio'] = towns[before] / towns[bottom]
    towns = towns[[before, bottom, 'Ratio']]

    
    # University counties and states. Find housing data by comparing overlap in list of all towns.
    univ = get_list_of_university_towns()
    uni = univ.merge(towns, left_on=['State', 'RegionName'], right_index=True, how='inner', indicator=True).dropna()

    # Non-university states and counties. Find housing data by eliminating university town data (initially merge saves overlap)
    non = univ.merge(towns, left_on=['State', 'RegionName'], right_index=True, how='right', indicator=True)
    non = non.query('_merge == "right_only"').dropna()

    
    # T-test for mean prices ratio:
    ## Ho: university == non-university
    ## Ha: university != non-university
    alpha = 0.01
    t, p = ttest_ind(uni['Ratio'], non['Ratio'])
    
    # Determine whether the mean price ratio between university towns is different (reject null) or not (fail to reject null)
    different = True if p <= 0.01 else False
    # Determine whether university towns have a lower or equivalent mean housing prices compared to non-university towns
    better = 'university town' if uni['Ratio'].mean() <= non['Ratio'].mean() else 'non-university town'

    
    # Conclusion: 
    result = "Since {} is less than our significance level of {}, we reject the null hypothesis. We can conclude that "
    statement = "the mean housing prices of university towns are less affected than non-university towns during a recession."
    print(result.format(p, alpha) + "\n" + statement)
    
    return (different, p, better)
run_ttest()

Since 0.0027240637047611684 is less than our significance level of 0.01, we reject the null hypothesis. We can conclude that 
the mean housing prices of university towns are less affected than non-university towns during a recession.


(True, 0.0027240637047611684, 'university town')